In [27]:
import yfinance as yf
import pandas as pd
from gdeltdoc import Filters, GdeltDoc
import re


def get_data_yfinance(quotation: str, start_date: str, end_date: str, interval='1d') -> pd.DataFrame:
    """Взять данные с yahoo finance
    params:
      quotation: название котировки, данные для которой хотим получить
      start_date, end_date - интервал, формат "год-месяц-день"
      interval - периодичность, формат "(номер)(первая буква слова (d, m, y))"
    returns:
      DataFrame формата "Тикет, Время, 6 видов цен"
    """
    df_res = yf.download(tickers=quotation,
                         start=start_date,
                         end=end_date,
                         interval=interval)
    df_res.loc[:, 'Ticker'] = quotation
    df_res = df_res.groupby(pd.Grouper(level="Date", freq=interval.upper())).mean()
    # Приводим время к одному виду для слияния
    df_res.index = pd.to_datetime(df_res.index).tz_localize('Etc/UCT')
    return df_res


def get_data_gdelt(quotation: str, keywords: list, start_date: str, end_date: str, interval="1d", num_records=250,
                   repeats=3) -> pd.DataFrame:
    """Взять данные с gdelt
    params:
      quotation - имя ценной бумаги
      keywords - из графа знаний по ключевому слову
      start_date, end_date - интервал, формат "год-месяц-день"
      interval - периодичность, формат "(номер)(первая буква слова (d, m, y))"
      num_records - сколько максимум записей взять за промежуток
      (не реализовано) repeats - сколько раз должно повториться ключевое слово в статье
    returns:
      DataFrame формата "Datetime (индекс), Ticker,
        [Average_Tone, Article_Count, Volume_Intensity]_[std, mean, sum, min, max]"
      Ticker и есть ключевое слово
    """

    df_res = None
    df_dub = pd.DataFrame()
    # Как называются колонки в полученных DataFrames
    col_names = ['Average_Tone', 'Article_Count', 'Volume_Intensity']

    # Что будем искать для данных ключевых слов
    # Тон статей, их количество и их кол-во в отношении ко всем остальным
    match_list = ["timelinetone", "timelinevolraw", "timelinevol"]
    match_dict = dict(zip(match_list, col_names))
    for ft in keywords:
        f = Filters(
            start_date=start_date,
            end_date=end_date,
            num_records=num_records,
            keyword=ft
        )

        gd = GdeltDoc()
        # Get a timeline of the number of articles matching the filters
        for timeline in match_list:
            timeline_data = gd.timeline_search(timeline, f)
            timeline_data = timeline_data.fillna(0)
            timeline_data = timeline_data.groupby(pd.Grouper(key="datetime", freq=interval.upper()))
            if timeline in ['timelinetone']:
                timeline_data = timeline_data.mean()
            else:
                timeline_data = timeline_data.sum()

            # Собираем все фичи в один датафрейм, далее их разделим
            col_name = match_dict[timeline]
            df_dub[f"{ft}_{timeline}_{col_name}"] = timeline_data[col_name.replace('_', ' ')].values

            # Так как мы копируем только колонки, то надо один раз откопировать дату в итог
            if df_res is None:
                # Так же выровняем индексы, чтобы при копировании не выдавалось NaN
                df_res = pd.DataFrame(index=timeline_data.index)
                df_dub.index = timeline_data.index

    # Нужно создать колонки со средним, средним отклонением, минимумом и максимумом для каждой фичи
    # Для начала нужно найти все колонки, для которых будем это считать
    columns = df_dub.columns

    # Сначала сформируем список датафреймов, которые нам нужно достать для каждой колонки
    for pattern in col_names:
        pattern_list = list()
        for col in columns:
            if pattern in col:
                pattern_list.append(col)

        # Теперь для pattern у нас есть список
        # Посчитаем для неё std, mean, min, max, sum
        df_res[f'{pattern}_min'] = df_dub[pattern_list].min(axis=1, skipna=True)
        df_res[f'{pattern}_max'] = df_dub[pattern_list].max(axis=1, skipna=True)
        df_res[f'{pattern}_mean'] = df_dub[pattern_list].mean(axis=1, skipna=True)
        df_res[f'{pattern}_std'] = df_dub[pattern_list].std(axis=1, skipna=True)
        df_res[f'{pattern}_sum'] = df_dub[pattern_list].sum(axis=1, skipna=True)

    df_res.loc[:, 'Ticker'] = quotation
    return df_res


def get_dataframe(**kwargs) -> pd.DataFrame:
    """ Получить полный датафрейм с источников
        params:
          quotation - имя ценной бумаги
          keywords - из графа знаний по ключевому слову
          start_date, end_date - интервал, формат "год-месяц-день"
          interval - периодичность, формат "(номер)(первая буква слова (d, m, y))"
          num_records - сколько максимум записей взять за промежуток
          (не реализовано) repeats - сколько раз должно повториться ключевое слово в статье
        returns:
          DataFrame формата "Datetime (индекс), Ticker,
            [Average_Tone, Article_Count, Volume_Intensity]_[std, mean, sum, min, max], - из новостей
            Open, High, Low, Close, Adj Close, Volume - из финансов
    """
#     pattern = re.compile('^(19|20)\d\d[-](0[1-9]|1[012])[-](0[1-9]|[12][0-9]|3[01])$')
#     assert pattern.match(kwargs['start_date']) is None, "Дата старта в правильном формате имеет вид (гггг-мм-дд)"
#     assert pattern.match(kwargs['end_date']) is None, "Дата конца в правильном формате имеет вид (гггг-мм-дд)"

    gdelt_data = get_data_gdelt(**kwargs)
    yfinance_data = get_data_yfinance(quotation=kwargs['quotation'],
                                      start_date=kwargs['start_date'],
                                      end_date=kwargs['end_date'],
                                      interval="1d" if not kwargs.get('interval') else kwargs['interval'])
    res = pd.concat([gdelt_data, yfinance_data], axis=1)
    del gdelt_data
    del yfinance_data
    return res

In [323]:
def prepare_train_samp(quotation_tr: str, keywords_tr: list, start_date_tr: str, end_date_tr: str,
                   features: list, size_window: int, days_forward: int) -> pd.DataFrame:
    #size_window - размер скользяцего окна
    #check - через сколько дней хотим проверить
    #features(названия брать из get_dataframe) - лист признаков которые нужны в итоге
    
    
    df = get_dataframe(quotation=quotation_tr, keywords=keywords_tr,start_date= start_date_tr, end_date=end_date_tr, interval="1d", num_records=250)
    #Выбираем тоьлко интересующие нас признаки
    df = df[features]
    
    #Дропаем наны
    df = df.dropna() 
    
    #df.query("Ticker == 'quotation_tr'").iloc[-size_window:-1].Open.to_list() + ['quotation_tr']
#     df.query("Ticker == 'quotation_tr'").iloc[-1].Open
    features.remove('Ticker')
    #Пробегаем по всем признакам
    for sign in features:
    #Смотрим предыдущие значения и слепляем их в наш Dataframe по тикету
        out = []
        for name in df.Ticker.unique():
            tmp = df.loc[df.query(f'Ticker == "{name}"').index]
            for i in range(-size_window, 0):
                column = f'{sign}_pr{i}'
                tmp[column] = tmp[f'{sign}'].shift(-i)
            out.append(tmp)
            df = pd.concat(out)
            
            
    df.loc[:, 'target'] = (df['Open'] - df['Open'].shift(periods=-days_forward))
    df.loc[:, 'target'] = df['target'].apply(lambda x: 1 if x < 0 else 0)
    #Дропаем наны которые появились из-за окна
    df.dropna(inplace=True)
    return df    


In [324]:
#Пример вызова
df = prepare_train_samp(quotation_tr='MSFT', keywords_tr=['microsoft'],
                        start_date_tr= '2017-01-01', end_date_tr='2021-01-10',
                        features =['Ticker', 'High', 'Open', 'Close', 'Average_Tone_sum'], 
                        size_window=3, days_forward=1)




[*********************100%***********************]  1 of 1 completed


In [325]:

df

,Ticker,High,Open,Close,Average_Tone_sum,High_pr-3,High_pr-2,High_pr-1,Open_pr-3,Open_pr-2,Open_pr-1,Close_pr-3,Close_pr-2,Close_pr-1,Average_Tone_sum_pr-3,Average_Tone_sum_pr-2,Average_Tone_sum_pr-1,target
datetime,,,,,,,,,,,,,,,,,,
2017-01-06 00:00:00+00:00,MSFT,63.150002,62.299999,62.840000,0.8981,62.840000,62.750000,62.660000,62.790001,62.480000,62.189999,62.580002,62.299999,62.299999,0.5453,1.0115,1.2107,1
2017-01-09 00:00:00+00:00,MSFT,63.080002,62.759998,62.639999,0.9374,62.750000,62.660000,63.150002,62.480000,62.189999,62.299999,62.299999,62.299999,62.840000,1.0115,1.2107,0.8981,0
2017-01-10 00:00:00+00:00,MSFT,63.070000,62.730000,62.619999,0.8288,62.660000,63.150002,63.080002,62.189999,62.299999,62.759998,62.299999,62.840000,62.639999,1.2107,0.8981,0.9374,0
2017-01-11 00:00:00+00:00,MSFT,63.230000,62.610001,63.189999,-0.1810,63.150002,63.080002,63.070000,62.299999,62.759998,62.730000,62.840000,62.639999,62.619999,0.8981,0.9374,0.8288,1
2017-01-12 00:00:00+00:00,MSFT,63.400002,63.060001,62.610001,0.4675,63.080002,63.070000,63.230000,62.759998,62.730000,62.610001,62.639999,62.619999,63.189999,0.9374,0.8288,-0.1810,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-04 00:00:00+00:00,MSFT,223.000000,222.529999,217.690002,-0.8388,227.179993,225.630005,223.000000,226.309998,225.229996,221.699997,224.149994,221.679993,222.419998,-0.1233,-0.1066,-0.8140,0
2021-01-05 00:00:00+00:00,MSFT,218.520004,217.259995,217.899994,-0.4331,225.630005,223.000000,223.000000,225.229996,221.699997,222.529999,221.679993,222.419998,217.690002,-0.1066,-0.8140,-0.8388,0
2021-01-06 00:00:00+00:00,MSFT,216.490005,212.169998,212.250000,-0.5907,223.000000,223.000000,218.520004,221.699997,222.529999,217.259995,222.419998,217.690002,217.899994,-0.8140,-0.8388,-0.4331,1
